In [104]:
!pip install sktime

In [105]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from sklearn.model_selection import train_test_split, StratifiedKFold
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV

In [106]:
rs=121
stocks=['aapl','googl','nvda','dal','xom','cvx','vz']

In [107]:
init_dir_path = str(os.getcwd())
data=pd.read_csv(init_dir_path+'/stock_hmm_output.csv',engine='python').drop(['Unnamed: 0'],axis=1)
features=['Return','Volatility','HighR','LowR']
target='State'
train, test = train_test_split(data,test_size=0.2,shuffle=True,random_state=rs,stratify=data['State'])

In [108]:
#Without TimeSeries
accuracy=0
ns=5
kfold=StratifiedKFold(n_splits=ns,shuffle=True,random_state=121)
for train_i, test_i in kfold.split(train,train[target]):
    training_set=train.iloc[train_i]
    testing_set=train.iloc[test_i]
    #Do training here for each model on training_set


    #Measure accuracy of the model on testing_set (using accuracy_score should be fine)


    #Add the accuracy scores from the multiple iterations

#Average the accuracy
ns=ns/5


In [109]:
#With TimeSeries
#First Set Aside Testing data
final_test=[]
l=[]
ft_size=400
for x in data.groupby(['Ticker']):
    final_test.append(x[1][-ft_size:])
    l.append(x[1][:-ft_size])
#Model Training
#model=
#model.fit(x=train_x,y=train_y)
#Model Prediction and Accuracy
#accuracy_score(test_y,model.predict(test_x))


In [110]:
def createSlidingWindow(tr,te,f,t):
    splitting=SlidingWindowSplitter(window_length=7,step_length=1,fh=[1])
    train_x=[]
    train_y=[]
    test_x=[]
    test_y=[]
    
    for i in splitting.split(tr):
        train_x.append(tr.iloc[i[0]][f].values.flatten())
        train_y.append(tr.iloc[i[1]][t].values.flatten()[0])
    
    for i in splitting.split(te):
        test_x.append(te.iloc[i[0]][f].values.flatten())
        test_y.append(te.iloc[i[1]][t].values.flatten()[0])
    return ((train_x,train_y),(test_x,test_y))

In [111]:
testing_size=500
ns=5
tss=TimeSeriesSplit(n_splits=ns,test_size=testing_size)
training_list=[]
testing_list=[]
for i in range(ns):
    training_list.append([[],[]])
    testing_list.append([[],[]])
for x in l:
    for (i,t) in enumerate(tss.split(x)):
        ((train_x,train_y),(test_x,test_y))=createSlidingWindow(x.iloc[t[0]],x.iloc[t[1]],features,target)
        training_list[i][0] += train_x
        training_list[i][1] += train_y
        testing_list[i][0] += test_x
        testing_list[i][1] += test_y


In [112]:
model_names=['ada','decisiontree']
accuracies=[]
models=[AdaBoostClassifier(n_estimators=100,random_state=121),DecisionTreeClassifier(random_state=rs)]
#Add Possible AdaBoostModels:
for m in models:
    a=0
    for i in range(ns):
        m.fit(X=training_list[i][0],y=training_list[i][1])
        a+= accuracy_score(m.predict(testing_list[i][0]),testing_list[i][1])
    a=a/ns
    accuracies.append(a)

In [120]:
a=0
for i in range(ns):
    a+=accuracy_score([1]*len(testing_list[i][1]),testing_list[i][1])
a=a/ns
print(a)

0.7711388003477253


In [117]:
accuracies

[0.8718052738336715, 0.7982613735149232]